In [ ]:
# Importing re package for using regular expressions 
import re
import datetime
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os.path

# Download the csv file from Mitre. Future enhancement could be to compare checksum to determine whether to download new version.
wgetcvedownload = 'https://cve.mitre.org/data/downloads/allitems.csv'

if os.path.isfile('/home/ec2-user/SageMaker/notebooks/tools/allitems.csv'):
    print("File already exists. No need to download.")
else:
    print('Need to download')
    ! wget {wgetcvedownload}
    ! mv 'allitems.csv' tools/

In [ ]:
dfCVE = pd.read_csv('tools/allitems.csv', sep=',', skiprows=[0,1,3,4,5,6,7,8,9], header=0, encoding = "ISO-8859-1")

# Function to clean the names - there is probably a better way of doing all of this.
def normalize_columns(Phase): 
    Phase = str(Phase)
    if re.search('(\()(\d\d\d\d\d\d\d\d)', Phase): 
        pos = re.search('(\()(\d\d\d\d\d\d\d\d)', Phase)
        pos = pos.group(2)
        date_str = pos
        format_str = '%Y%m%d'
        pos = datetime.datetime.strptime(date_str, format_str)      
        return pos
    else: 
        # if cleanup needed return the same name 
        pos = "19000101"
        date_str = pos
        format_str = '%Y%m%d'
        pos = datetime.datetime.strptime(date_str, format_str) 
        return 'null'

# Add a normalized CVE date column
dfCVE['CVEDate'] = dfCVE['Phase'].apply(normalize_columns)
# Drop the null date values
dfCVE.drop(dfCVE[dfCVE.CVEDate == 'null'].index, inplace=True)
#dfCVE.drop(dfCVE[dfCVE.Status == 'Candidate'].index, inplace=True)
#dfCVE.drop(dfCVE[dfCVE.Description  'Reserved'].index, inplace=True)
dfCVE = dfCVE[~dfCVE.Description.str.contains("RESERVED")]
dfCVE

In [ ]:
# How many per year
CVEYear = dfCVE['CVEDate'].groupby([dfCVE.CVEDate.dt.year]).agg('count').to_frame('counts')
# Convert results to dataframe
dfCVEYear = pd.DataFrame(CVEYear)
dfCVEYear = dfCVEYear.reset_index()
dfCVEYear.columns = ['Year', 'Count'] # change column names
dfCVEYear

In [ ]:
# Which months are busiest for CVE - Could indicate more patching
CVEMonth = dfCVE['CVEDate'].groupby([dfCVE.CVEDate.dt.month]).agg('count').to_frame('counts')
# Convert results to dataframe
dfCVEMonth = pd.DataFrame(CVEMonth)
dfCVEMonth = dfCVEMonth.reset_index()
dfCVEMonth.columns = ['Month', 'Count'] # change column names
dfCVEMonth

In [ ]:
import numpy as np
from bokeh.io import output_notebook, show
from bokeh.plotting import figure

# Confirm that Bokeh is working
output_notebook()
dfCVEYearMod = dfCVEYear.iloc[1:]
p = figure(plot_width=400, plot_height=400)
p.circle(dfCVEYearMod['Year'],dfCVEYearMod['Count'], size=10, line_color="navy", fill_color="red", fill_alpha=0.5)

show(p) # show the results